# GCN on MUTAG dataset

code taken from: 
- https://colab.research.google.com/drive/1mMUKnvM_Byu8wEcJpFSYGnniPPhIOD7N?usp=sharing#scrollTo=f4sXdEnBOVke
- https://theaisummer.com/graph-convolutional-networks/

In [54]:
! pip install ipynb
! pip install torch

In [55]:
!pip install torchnet networkx

In [56]:
import ipynb.fs.full.basics_laplace as basics_laplace
import torch

# Data preparation

In [57]:
import torchnet as tnt
import os
import networkx as nx
import numpy as np
import torch

In [58]:
def indices_to_one_hot(number, nb_classes, label_dummy=-1):
    """Convert an iterable of indices to one-hot encoded labels."""
    if number == label_dummy:
        return np.zeros(nb_classes)
    else:
        return np.eye(nb_classes)[number]

def get_graph_signal(nx_graph):
    d = dict((k, v) for k, v in nx_graph.nodes.items())
    x = []
    invd = {}
    j = 0
    for k, v in d.items():
        x.append(v['attr_dict'])
        invd[k] = j
        j = j + 1
    return np.array(x)


def load_data(path, ds_name, use_node_labels=True, max_node_label=10):
    node2graph = {} # dictionary which node belongs to which graph
    Gs = [] # list of all the graphs
    data = []
    dataset_graph_indicator = f"{ds_name}_graph_indicator.txt"
    dataset_adj = f"{ds_name}_A.txt"
    dataset_node_labels = f"{ds_name}_node_labels.txt"
    dataset_graph_labels = f"{ds_name}_graph_labels.txt"

    path_graph_indicator = os.path.join(path, dataset_graph_indicator)
    path_adj = os.path.join(path, dataset_adj)
    path_node_lab = os.path.join(path, dataset_node_labels)
    path_labels = os.path.join(path, dataset_graph_labels)


    #
    # (2) DS_graph_indicator.txt (n lines)
    # column vector of graph identifiers for all nodes of all graphs,
    # the value in the i-th line is the graph_id of the node with node_id i
    #
    # TL;DR: we have a list of (node_id, graph_id), we want to build graph objects and assign the node-objects to them
    #
    
    with open(path_graph_indicator, "r") as f:
        c = 1
        for line in f: # c is the node id, line is the id of the graph it belongs to
            node2graph[c] = int(line[:-1])
            if not node2graph[c] == len(Gs): # create a new graph if the line specifies a unseen graph
                Gs.append(nx.Graph()) # add it to list of graphs
            Gs[-1].add_node(c) # add the node to the graph
            c += 1

    #(1) DS_A.txt (m lines) 
    # sparse (block diagonal) adjacency matrix for all graphs,
    # each line corresponds to (row, col) resp. (node_id, node_id)
    #
    # TL;DR: each line is (node_id, node_id), specifies an edge between two nodes 
    with open(path_adj, "r") as f:
        for line in f:
            edge = line[:-1].split(",")
            edge[1] = edge[1].replace(" ", "")
            Gs[node2graph[int(edge[0])] - 1].add_edge(int(edge[0]), int(edge[1]))

            
    # (4) DS_node_labels.txt (n lines)
    # column vector of node labels,
    # the value in the i-th line corresponds to the node with node_id i
    # 
    # TL;DR: the line-index is the node_id, the line itself is the index of the node type. convert this index to one-hot
    if use_node_labels:
        with open(path_node_lab, "r") as f:
            c = 1
            for line in f:
                node_label = indices_to_one_hot(int(line[:-1]), max_node_label)
                Gs[node2graph[c] - 1].add_node(c, attr_dict=node_label)
                c += 1

                
    #(3) DS_graph_labels.txt (N lines) 
    # class labels for all graphs in the dataset,
    #the value in the i-th line is the class label of the graph with graph_id i
    #
    # TL;DR the i-th line contains the label for the i-th graph
    labels = []
    with open(path_labels, "r") as f:
        for line in f:
            labels.append(int(line[:-1]))

    # two lists, one containing the graphs, one containing the graph-labels
    return list(zip(Gs, labels)) 

def create_loaders(dataset, batch_size, split_id, offset=-1):
    train_dataset = dataset[:split_id]
    val_dataset = dataset[split_id:]
    return to_pytorch_dataset(train_dataset, offset,batch_size), to_pytorch_dataset(val_dataset, offset,batch_size)

def to_pytorch_dataset(dataset, label_offset=0, batch_size=1):
    list_set = []
    for graph, label in dataset:
        F, G = get_graph_signal(graph), nx.to_numpy_matrix(graph)
        numOfNodes = G.shape[0]
        F_tensor = torch.from_numpy(F).float()
        G_tensor = torch.from_numpy(G).float()

        # fix labels to zero-indexing
        if label == -1:
            label = 0
    
        label += label_offset
    
        list_set.append(tuple((F_tensor, G_tensor, label)))

    dataset_tnt = tnt.dataset.ListDataset(list_set)
    data_loader = torch.utils.data.DataLoader(dataset_tnt, shuffle=True, batch_size=batch_size)
    return data_loader



dataset = load_data(path='./MUTAG/', ds_name='MUTAG',
                  use_node_labels=True, max_node_label=7)
train_dataset, val_dataset = create_loaders(dataset, batch_size=1, split_id=150, offset=0)
print('Data are ready')

Data are ready


## Implementing a 1-hop GCN layer in Pytorch

Y is the output. Lnorm is the Laplacian of the Adjacency Matrix A. X is the graph signal. W are the learnable parameters. D is the Degree-Matrix of A. I is the Identity Matrix.

$$Y=Lnorm​XW$$
$$L^{norm}mod​=D^{−1/2}​(A+I)D^{-1/2}​$$

In [59]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F


In [60]:
# run something on specific hardware, e.g. cuda
def device_as(x,y):
    return x.to(y.device)

def calc_degree_matrix_norm(a):
    return torch.diag_embed(torch.pow(a.sum(dim=-1),-0.5))

def create_graph_lapl_norm(a):
    size = a.shape[-1]
    a +=  device_as(torch.eye(size),a)
    D_norm = calc_degree_matrix_norm(a)
    L_norm = torch.bmm( torch.bmm(D_norm, a) , D_norm )
    return L_norm

In [61]:
# class GCN_AISUMMER extends nn.Module
class GCN_AISUMMER(nn.Module):
    """
    A simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()
        # Applies a linear transformation to the incoming data: y=x*A^(T)+b*y
        self.linear = nn.Linear(in_features, out_features, bias=bias) 
        

    def forward(self, X, A):
        """
        A: adjecency matrix
        X: graph signal
        """
        L = create_graph_lapl_norm(A)
        x = self.linear(X)
        return torch.bmm(L, x)

In [62]:
# class GNN extends torch.nn.Module
class GNN(nn.Module):
    def __init__(self,
                    in_features = 7, # cora dataset has 7 different topics
                    hidden_dim = 64,
                    classes = 2,
                    dropout = 0.5):
        super(GNN, self).__init__()

        self.layers = []
        self.layers.append(GCN_AISUMMER(in_features, hidden_dim))
        self.layers.append(GCN_AISUMMER(hidden_dim, hidden_dim))
        self.layers.append(GCN_AISUMMER(hidden_dim, hidden_dim))
        self.fc = nn.Linear(hidden_dim, classes)
        self.dropout = dropout

    def forward(self, x, A):
        for layer in self.layers:
            x = layer(x,A)
            x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        # aggregate node embeddings
        x = x.mean(dim=1)
        # final classification layer
        return self.fc(x)

In [63]:
criterion = torch.nn.CrossEntropyLoss()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f'Training on {device}')
model = GNN(in_features = 7,
                hidden_dim = 128,
                classes = 2).to(device)

optimizer= torch.optim.Adam(model.parameters(), lr=0.01)

def train(train_loader):
    model.train()

    for data in train_loader: 
        optimizer.zero_grad()  
        X, A, labels = data
        X, A, labels = X.to(device), A.to(device), labels.to(device)  
        # Forward pass.
        out = model(X, A)  
        # Compute the graph classification loss.
        loss = criterion(out, labels) 
        # Calculate gradients.
        loss.backward()  
        # Updates the models parameters
        optimizer.step() 

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        X, A, labels = data
        X, A, labels = X.to(device), A.to(device), labels.to(device)  
        # Forward pass.
        out = model(X, A)  
        # Take the index of the class with the highest probability.
        pred = out.argmax(dim=1) 
        # Compare with ground-truth labels.
        correct += int((pred == labels).sum()) 
    return correct / len(loader.dataset)  

best_val = -1
for epoch in range(1, 241):
    train(train_dataset)
    train_acc = test(train_dataset)
    val_acc = test(val_dataset)
    if val_acc>best_val:
        best_val = val_acc
        epoch_best = epoch
    
    if epoch%10==0:
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f} || Best Val Score: {best_val:.4f} (Epoch {epoch_best:03d}) ')
        
print('done training model!')

Training on cpu
Epoch: 010, Train Acc: 0.6600, Val Acc: 0.6842 || Best Val Score: 0.7105 (Epoch 003) 
Epoch: 020, Train Acc: 0.6733, Val Acc: 0.6842 || Best Val Score: 0.7105 (Epoch 003) 
Epoch: 030, Train Acc: 0.6667, Val Acc: 0.6842 || Best Val Score: 0.7105 (Epoch 003) 
Epoch: 040, Train Acc: 0.7000, Val Acc: 0.6842 || Best Val Score: 0.7105 (Epoch 003) 
Epoch: 050, Train Acc: 0.6933, Val Acc: 0.6579 || Best Val Score: 0.7105 (Epoch 003) 
Epoch: 060, Train Acc: 0.7067, Val Acc: 0.6842 || Best Val Score: 0.7105 (Epoch 003) 
Epoch: 070, Train Acc: 0.7267, Val Acc: 0.6579 || Best Val Score: 0.7105 (Epoch 003) 
Epoch: 080, Train Acc: 0.7200, Val Acc: 0.6842 || Best Val Score: 0.7105 (Epoch 003) 
Epoch: 090, Train Acc: 0.7267, Val Acc: 0.6842 || Best Val Score: 0.7105 (Epoch 003) 
Epoch: 100, Train Acc: 0.6867, Val Acc: 0.6842 || Best Val Score: 0.7105 (Epoch 003) 
Epoch: 110, Train Acc: 0.7200, Val Acc: 0.6842 || Best Val Score: 0.7368 (Epoch 106) 
Epoch: 120, Train Acc: 0.7133, Val Acc